In [3]:
from dotenv import load_dotenv
load_dotenv(".env.local")

True

## .docx to text

### AzureAI Document Intelligence

In [78]:
from langchain_community.document_loaders.doc_intelligence import (
    AzureAIDocumentIntelligenceLoader,
)
import os
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
def load_docs_adi(filepath: str):
    adi = AzureAIDocumentIntelligenceLoader(
        "https://eastus.api.cognitive.microsoft.com",
        AZURE_DOCUMENT_INTELLIGENCE_API_KEY,
        file_path=filepath,
        api_model="prebuilt-read",
    )
    docs = adi.load()
    return docs

### PyPDF

In [85]:
from langchain_community.document_loaders import PyPDFLoader

def load_docs_pypdf(file_path: str):
    pypdf_loader = PyPDFLoader(
        file_path = file_path,
        password = None,
        extract_images = True,
        # headers = None
        # extraction_mode = "plain",
        # extraction_kwargs = None,
    )
    docs = pypdf_loader.load()
    return docs

### Zerox

In [132]:
from langchain.schema import Document
def load_docs_zerox(path:str):
    with open(path, "r") as f:
        data = f.read()
    return [Document(page_content=data)]

In [133]:
current_dir = os.getcwd()
#file_path = os.path.join(current_dir, "RA Duty Manual - Final Draft 2023-2024.pdf")
file_path = os.path.join(current_dir, "./zerox/output.md")

docs = load_docs_zerox(file_path)

## Pinecone

In [7]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone.grpc import GRPCVector
from pinecone.grpc.index_grpc import UpsertResponse

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_HOST = os.getenv("PINECONE_HOST")
PINECONE_NAMESPACE = "umd-call-center"
pc = Pinecone(api_key=PINECONE_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
PC_INDEX_NAME = "knowledge"
pc_index = pc.Index(PC_INDEX_NAME, host=PINECONE_HOST)

In [ ]:
from typing import List
def query_pc(vector: List[float]):
    query_result = pc_index.query(
        vector=vector,
        namespace=PINECONE_NAMESPACE,
        top_k=10,
        #filter={"knowledge_uuid": {"$in": knowledge_uuids}},
        include_metadata=True,
        timeout=1,
    )
    return query_result


## Create Local Embeddings

In [134]:
from langchain_openai import OpenAIEmbeddings
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

embedder = OpenAIEmbeddings(
    model="text-embedding-3-large", api_key=OPENAI_API_KEY
)

In [135]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

def split_by_semantics(content:str):
    semantic_splitter = SemanticChunker(
        embedder,
        add_start_index=True,
        sentence_split_regex=r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)(?=\s)",
    )
    splits = semantic_splitter.split_documents([Document(page_content=content)])
    start_index = 0
    for i, split in enumerate(splits):
        last_index = start_index + len(content) - 1
        split.metadata["start_index"] = start_index
        start_index = last_index + 1
    return splits

def split_by_character(content:str):
    recursive_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800, chunk_overlap=400, add_start_index=True
    )
    splits = recursive_splitter.split_documents([Document(page_content=content)])
    return splits

## Generate Test Dataset

## Measuring

### Retrieval relevence
Are the documents that were retrieved relevent to the query

### Answer Faithfulness
Is the answer grounded in the documents?
- This can measure hallucinations

### Answer Correctness
Is the answer consistent with a reference answer?

### Code

#### Gen dataset giskard

In [136]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
splits = []
for doc in docs:
    splits.extend(split_by_character(doc.page_content))

In [137]:
vector_store = InMemoryVectorStore(OpenAIEmbeddings(model="text-embedding-3-large"))
vector_store.add_documents(splits)

['8aa6cf10-812f-48d8-8437-7cb0de6a4243',
 'be365dfb-f8c9-4b6f-a14b-c3966d0030f6',
 '122f53f0-5f27-41b5-9633-b2d95b1aa255',
 'c7f7e036-cd31-4451-a069-6facce129fc4',
 '031777de-28ab-42bb-adfd-ee31b11d5d82',
 '846b9aea-9354-49f5-a9f2-aa49898d2611',
 '89e572f4-cedd-4344-9948-da49e8fd9c52',
 'd7c99160-9bfe-4504-a202-f15ad1b0c917',
 '80e4da48-ed34-4ac0-8510-8f05df238c97',
 'a476cfc0-a78f-404c-8924-c037aef868d5',
 'a14f7877-7908-4a60-9637-8afafd922d91',
 'dc483b9d-2599-486f-b49f-2d7f093b94a9',
 '7c551313-4487-436f-8bb5-8a94d78374b6',
 '11ab8557-09ce-4c6d-8529-c091effd5f35',
 'aef5caba-f69f-498e-a2ec-a16d67766363',
 'c3f3d7e6-64c4-4ffd-8720-f33ee477c62f',
 '43b90a1e-6879-4ce9-8b47-4fadc62c33d0',
 'be12fb19-56fe-4e73-958c-90e6ba39cbce',
 '1410d79f-43a5-44e1-ab00-477d7e48fc6e',
 '98d9c228-4ba5-4dfb-a651-324e69a503c2',
 '24ebf26d-f6d4-4207-b58c-6797e6650761',
 '3bb3986d-b857-4ba5-89b3-b376e2e66f93',
 'e387d3e5-f08c-4591-858f-b3769ea6196e',
 '90972e69-6e18-4c63-9a08-8c1ef86881ed',
 'd715e570-b012-

In [138]:
import pandas as pd
from giskard.rag import KnowledgeBase
knowledge_base_df = pd.DataFrame([i.page_content for i in splits], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)

In [139]:
import giskard
from giskard.llm.client.openai import OpenAIClient

giskard.llm.set_llm_api("openai")
oc = OpenAIClient(model="gpt-4o-mini")
giskard.llm.set_default_client(oc)

In [119]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=25,
    language='en',  # optional, we'll auto detect if not provided
    agent_description="A support agent for the University of Maryland's call center", # helps generating better questions
)

2024-10-27 17:19:51,625 pid:49470 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2024-10-27 17:19:59,011 pid:49470 MainThread giskard.rag  INFO     Found 5 topics in the knowledge base.


Generating questions: 100%|██████████| 25/25 [00:47<00:00,  1.91s/it]


In [123]:
testset.save("testset25.jsonl")

In [140]:
from giskard.rag import QATestset
testset = QATestset.load("testset25.jsonl")

In [141]:
testset_df = testset.to_pandas()
testset_df.head()

,question,reference_answer,reference_context,conversation_history,metadata
id,,,,,
b0c34dd1-5373-4a31-b941-b7672d4345bb,What should a Resident Assistant do if there i...,Contact UMPD immediately. RAs should NOT try t...,Document 173: Misconduct (OCRSM): (301)405-114...,[],"{'question_type': 'simple', 'seed_document_id'..."
0e624b2c-5816-4b41-a857-70ec8e87661c,What should you do if there is a question abou...,"The person should not be left alone, and UMPD ...",Document 73: Resources● CommunityLivingHandboo...,[],"{'question_type': 'simple', 'seed_document_id'..."
b1a6b840-1519-4eeb-849d-7bdc169bb22b,What is the policy regarding pets in the housi...,Only fish in aquariums no larger than 10 gallo...,Document 100: Resources● HousingAccommodations...,[],"{'question_type': 'simple', 'seed_document_id'..."
3a162c15-735d-415e-8ce6-52cfe8439388,What is the purpose of UMDAlert?,The context does not provide specific informat...,Document 6: 37SevereWeather 37EarlyWarningSyst...,[],"{'question_type': 'simple', 'seed_document_id'..."
0e4454d5-5f36-4e1d-9f31-2d7c4af1d6db,What should be documented according to the pro...,"Any vandalism, safety, and facility issues sho...","Document 32: Proppeddoors, fireextinguisher, f...",[],"{'question_type': 'simple', 'seed_document_id'..."


#### Evaluate gkiskard

In [147]:
from giskard.rag import evaluate
import openai

# Wrap your RAG model
def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    # Format appropriately the history for your RAG agent
    messages = history if history else []
    messages.append({"role": "user", "content": question})

    system_message = "Search results found the following information that might be relevent: "
    similarity_search_with_score = vector_store.similarity_search_with_score(question, k=10)
    for i, doc_score in enumerate(similarity_search_with_score):
        doc, score = doc_score
        system_message += f"Source {i}. Relevency Score: {score}:\n"+ doc.page_content + "\n\n"
        print(score)
    
    messages.append({"role": "system", "content": system_message})

    reminder_message = "Remember, you are on a phone call. Your response to the caller should be accurate and concise. Do not monologue. Here is the caller's message:"
    messages.append({"role": "system", "content": reminder_message})

    chatcompletion = openai.chat.completions.create(messages=messages, model="gpt-4o-mini")
    answer = chatcompletion.choices[0].message.content
    return answer


#ragas_context_recall = RagasMetric(name="RAGAS Context Recall", metric=context_recall, requires_context=True)

report = evaluate(get_answer_fn, testset=testset, knowledge_base=knowledge_base)

Asking questions to the agent:   0%|          | 0/25 [00:00<?, ?it/s]

0.7237126982455698
0.6514098405948775
0.6145697745967713
0.6068658810293511
0.6025936099001936
0.5964764142498462
0.5902256182761509
0.589838316419847
0.5706793613001738
0.5648190586039237


Asking questions to the agent:   4%|▍         | 1/25 [00:03<01:31,  3.81s/it]

0.5757888117884399
0.548253871219513
0.5444709183453731
0.5430358268789178
0.5319537219203614
0.5314839334522206
0.5004866809664246
0.4913180445759836
0.49127649762556236
0.4909809232390961


Asking questions to the agent:   8%|▊         | 2/25 [00:06<01:17,  3.37s/it]

...

Asking questions to the agent:  92%|█████████▏| 23/25 [00:50<00:03,  1.68s/it]

0.23167218549473592
0.20645081263592188
0.20206284412556813
0.1898131430756027
0.1847420402141643
0.17775405498393237
0.17552635266009825
0.1719223966582029
0.17158996281737926
0.16813970899058298


Asking questions to the agent:  96%|█████████▌| 24/25 [00:51<00:01,  1.46s/it]

0.18161314831851225
0.17042193190307853
0.16948247714615225
0.16905574540889326
0.16842111768154575
0.16820469846014835
0.1675828702922835
0.16524818848727882
0.1601726204868421
0.1584932845721518


CorrectnessMetric evaluation: 100%|██████████| 25/25 [00:21<00:00,  1.17it/s]


In [ ]:
#display(report)  # if you are working in a notebook

# or save the report as an HTML file
#report.to_html("rag_eval_report.html")

In [148]:
report.save("reports/gpt_4o_mini-splits_character-top10-zerox")

In [ ]:
from giskard.rag import RAGReport
loaded_report = RAGReport.load("reports/...")

In [65]:
report._knowledge_base._to_

In [63]:
report.failures

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
9c63fed4-b956-4aa0-9501-741321f4f034,What happens if a student does not work throug...,"If a student does not work through Labor Day, ...",Document 147: summer benefits\nSummer housing ...,[],"{'question_type': 'simple', 'seed_document_id'...",If a student living in summer housing does not...,False,The agent provided a general response about im...
f29be739-0393-44a5-a519-7912dba4c9e1,What should a staff member do if they are goin...,They are expected to immediately call the Serv...,Document 45: If a staff member is unable to re...,[],"{'question_type': 'simple', 'seed_document_id'...",If a staff member knows they will be late for ...,False,The agent provided a general response about no...
fb530e2e-e211-41f1-9c11-3e5bc8078666,In order to be considered for a promotion to t...,An employee must work a minimum of 120 total h...,Document 135: a. Possess a satisfactory perfor...,[],"{'question_type': 'complex', 'seed_document_id...",The minimum total number of hours an employee ...,False,The agent did not provide the specific minimum...
c9e0790a-3be2-4adf-b9bd-7de3f0c01105,Could you specify the individuals who hold the...,"Only the Service Center Assistant Manager, Man...","Document 162: All keys, swipes and key rings i...",[],"{'question_type': 'complex', 'seed_document_id...",The authority to approve the signing out of ke...,False,The agent provided a broader list of roles tha...
94e282d7-352b-42f9-b78e-762e24a61c1d,What steps should a student take regarding the...,"If you have a student parking permit, remove i...",Document 55: If you have a student parking per...,[],"{'question_type': 'distracting element', 'seed...",If a student has received a written warning fo...,False,The agent provided a detailed response about g...
e049d396-2658-4b2f-8722-e42697cdc731,In the event that a substitute fails to report...,The original (scheduled/assigned) staff member...,Document 48: If the substitute does not report...,[],"{'question_type': 'distracting element', 'seed...",When a substitute fails to report for their sc...,False,The agent provided a detailed response about t...
14060525-e7cd-4fa3-b239-501a78dc994c,"Hi, I'm a staff member at the university and I...",The driver should take the vehicle out to Moto...,Document 171: Vehicles should never be returne...,[],"{'question_type': 'situational', 'seed_documen...",It's important to ensure that university vehic...,False,The agent provided general refueling procedure...
42d9bf93-4e26-4fef-b301-c31e7cd69bd8,"Hi there, I’m currently dealing with an unexpe...",If a staff member is not able to safely travel...,Document 63: In the event the University has a...,[],"{'question_type': 'situational', 'seed_documen...",If you are a staff member unable to safely tra...,False,The agent provided a detailed response about g...
ad2c7d12-7bb1-4710-9efc-16f83d2bbbeb,What is the main purpose of the Service Center...,The Service Center serves as the main communic...,Document 6: The Department of Residential Faci...,[],"{'question_type': 'double', 'original_question...",The main purpose of the Service Center is to p...,False,The agent provided a general description of th...


## Misc 

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone.grpc import GRPCVector
from pinecone.grpc.index_grpc import UpsertResponse

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_HOST = os.getenv("PINECONE_HOST")
PINECONE_NAMESPACE = "umd-call-center"
pc = Pinecone(api_key=PINECONE_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
PC_INDEX_NAME = "knowledge"
pc_index = pc.Index(PC_INDEX_NAME, host=PINECONE_HOST)

In [ ]:
from typing import List
def query_pc(vector: List[float]):
    query_result = pc_index.query(
        vector=vector,
        namespace=PINECONE_NAMESPACE,
        top_k=10,
        #filter={"knowledge_uuid": {"$in": knowledge_uuids}},
        include_metadata=True,
        timeout=1,
    )
    return query_result


In [ ]:
import numpy as np

def cosine_similarity(vec1, vec2):
    # Compute dot product
    dot_product = np.dot(vec1, vec2)
    # Compute magnitudes
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    # Calculate cosine similarity
    if magnitude_vec1 == 0 or magnitude_vec2 == 0:
        return 0  # Handle the case of zero magnitude
    return dot_product / (magnitude_vec1 * magnitude_vec2)
